In [ ]:
from PIL import Image
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import math
import datetime

import caffe
from sklearn.cross_validation import train_test_split

# Segmented video location
set_1_folder = "../segmented_set1/"
set_1_files = os.listdir(set_1_folder)
height = 100
width = 100

resize = (height, width)

# Avi files have the following naming convention:
# file-id_sequence-number_activity-class

def get_frames_and_labels(f):
    cap = cv2.VideoCapture(f)
    frames = []
    label = int(f.split("_")[-1][0])
#     label = int(f.split("_")[2]) # This looks at sequence
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = np.array(Image.fromarray(frame.astype(np.uint8)).resize(resize, 
                                    Image.ANTIALIAS)) / 255.0
        frame = np.mean(frame, axis=2)        
        frames.append(frame)
    labels = [label for i in range(len(frames))]
    cap.release()
    cv2.destroyAllWindows()
    return frames, labels

from itertools import islice
group_adjacent = lambda a, k: zip(*(islice(a, i, None, k) for i in range(k)))

def group_frames(fname, vol_size):
    frames, labels = get_frames_and_labels(fname)
    label = labels[0]
    return_vols, return_labels = [], []
    grouped_frames = group_adjacent(frames, vol_size)
    print "Shrunk %i frames to %i sequences" % (len(frames), len(grouped_frames))
    for group in grouped_frames:
        vol = np.stack((group)).transpose((1, 2, 0))
        return_vols.append(vol)
        return_labels.append(label)
    final_vols = np.stack((return_vols))
    return final_vols, return_labels

def get_all_volumes_and_labels(f_store, vol_size=10):
    all_vols, all_labels = [], []
    for f in f_store:
        fname = set_1_folder + f
        vols, labels = group_frames(fname, vol_size)
        all_vols.append(vols)
        all_labels = all_labels + labels
    volumes = np.vstack((all_vols))
    labels = np.array(all_labels)
    print volumes.shape
    print labels.shape
    return volumes, labels

# set_1_frames = []
# set_1_labels = []
# for f in set_1_files:
#     frames, labels = get_frames_and_labels(set_1_folder + f)
#     set_1_frames = set_1_frames + frames
#     set_1_labels = set_1_labels + labels

In [ ]:
train, labels = get_all_volumes_and_labels(set_1_files)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.1)
print X_train.shape
print X_test.shape

In [ ]:
plt.figure(figsize=(20, 20))

to_plot = X_train[200]

for i in range(to_plot.shape[2] - 4):
    # Display original
    ax = plt.subplot(1, to_plot.shape[2], i + 1)
    plt.imshow(to_plot[:,:,i], cmap="Greys_r")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.tight_layout()

In [ ]:
vol = 10

def autoencoder(input_shape=[None, height, width, vol],
                n_filters=[3, 30, 50, 50],
                filter_sizes=[5, 10, 10, 10],
                corruption=False):
    # input to the network
    x = tf.placeholder(tf.float32, input_shape, name='x')

    # ensure 2-d is converted to square tensor.
    if len(x.get_shape()) == 2:
        x_dim = np.sqrt(x.get_shape().as_list()[1])
        if x_dim != int(x_dim):
            raise ValueError('Unsupported input dimensions')
        x_dim = int(x_dim)
        x_tensor = tf.reshape(
            x, [-1, x_dim, x_dim, n_filters[0]])
    elif len(x.get_shape()) == 4:
        x_tensor = x
    else:
        raise ValueError('Unsupported input dimensions')
    current_input = x_tensor

    # Build the encoder
    encoder_weights = []
    encoder_ops = []
    shapes = []
    for layer_i, n_output in enumerate(n_filters[1:]):
        n_input = current_input.get_shape().as_list()[3]
        shapes.append(current_input.get_shape().as_list())
        W = tf.Variable(
            tf.random_uniform([
                filter_sizes[layer_i],
                filter_sizes[layer_i],
                n_input, n_output],
                -1.0 / math.sqrt(n_input),
                1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder_weights.append(W)
        output = tf.nn.sigmoid(
            tf.add(tf.nn.conv2d(
                current_input, W, strides=[1, 1, 1, 1], padding='SAME'), b))
        encoder_ops.append(output)
        current_input = output

    # store the latent representation
    z = current_input
    encoder_weights.reverse()
    shapes.reverse()

    # Build the decoder using the same weights
    for layer_i, shape in enumerate(shapes):
        W = encoder_weights[layer_i]
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]))
        output = tf.nn.sigmoid(
            tf.add(tf.nn.conv2d_transpose(
                current_input, W,
                tf.pack([tf.shape(x)[0], shape[1], shape[2], shape[3]]),
                strides=[1, 1, 1, 1], padding='SAME'), b))
        current_input = output
        
    decoder = current_input

    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x_tensor))

    return {'x': x, 'z': z, 'y': y, 'cost': cost, 
            "encoder": encoder_ops, "decoder": decoder}


def test_ut(X_train, X_test, n_filters, filter_sizes):
    import tensorflow as tf
    ae = autoencoder(n_filters=n_filters, filter_sizes=filter_sizes)

    learning_rate = 0.01
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['cost'])

    # We create a session to use the graph
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.run(tf.initialize_all_variables())

    # Fit all training data
    batch_size = 100
    n_epochs = 50
    step_size = 10
    for epoch_i in range(n_epochs):
        for batch_i in range(X_train.shape[0] // batch_size):
            batch_xs = X_train[batch_i * batch_size:(batch_i + 1) * batch_size]
            train = batch_xs
            sess.run(optimizer, feed_dict={ae['x']: train})
        if epoch_i % step_size == 0:
            print(str(datetime.datetime.now()), epoch_i, sess.run(ae['cost'], feed_dict={ae['x']: train}))

    ae["session"] = sess
    
    return ae

In [ ]:
ae = test_ut(X_train, X_test,
                n_filters=[10, 10, 10, 5],
                filter_sizes=[3, 3, 3])

In [ ]:
train = X_train
combined = []
sess = ae["session"]
batch_size = 100
for batch_i in range(train.shape[0] // batch_size):
    batch_xs = train[batch_i * batch_size:(batch_i + 1) * batch_size]
    layers = [sess.run(ae["encoder"][i], 
            feed_dict={ae['x']: batch_xs}) for i in range(len(ae["encoder"]))]
    ravels = (np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"])))
    interm = np.hstack((ravels))
    combined.append(interm)

In [ ]:
combined = np.vstack((combined))
print combined.shape

In [ ]:
# Scale and visualize the embedding vectors
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=(20, 10))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 12})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

vectorized_imgs = combined

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(vectorized_imgs)

In [ ]:
plot_embedding(X_tsne, y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
knn.score(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(combined, y_train[:X_tsne.shape[0]])

In [ ]:
knn.score(combined, y_train[:X_tsne.shape[0]])

# Nearest neighbor querying

In [ ]:
from scipy.spatial import cKDTree
import random

test_idxes = random.sample(range(combined.shape[0]), 10)
print test_idxes

tree = cKDTree(combined)
test_array = combined[test_idxes]
query_res = tree.query(test_array, k=5)

nns = []

for idx, row in enumerate(query_res[1]):
    nn = X_train[row.ravel()]
#     to_plot = np.vstack([X_test[idx], nn])
    nns.append(nn)
    
nns = np.stack(nns)

In [ ]:
def plot_nearest_neighbors(nns, reshape=(100, 100)):
    fig, axs = plt.subplots(nns.shape[0], nns.shape[1], figsize=(32, 32))
    
    for i in range(nns.shape[0]):
        for j in range(nns.shape[1]):
            if len(reshape) != 2:
                axs[i][j].imshow(
                    nns[i, j][:,:,0].reshape(reshape))
            else:
                axs[i][j].imshow(
                    nns[i, j][:,:,0].reshape(reshape), cmap="Greys_r")
            axs[i][j].axis("off")
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()

In [ ]:
nns.shape

In [ ]:
plot_nearest_neighbors(nns)

In [ ]:
del ae

In [ ]:
def plot_nn_volumes(nns, reshape=(120, 120), row_examples=2):
    vol_size = 5
    skip = 2
    fig, axs = plt.subplots(row_examples * nns.shape[1], vol_size, figsize=(32, 64))
    
    for i in range(row_examples): # Number of row examples
        for j in range(nns.shape[1]):
            for k in range(vol_size): # Number of frames in the volume we want to see                
                if len(reshape) != 2:
                    axs[i * nns.shape[1] + j][k].imshow(
                        nns[i, j][:,:,k * skip].reshape(reshape))
                else:
                    axs[i * nns.shape[1] + j][k].imshow(
                        nns[i, j][:,:,k * skip].reshape(reshape), cmap="Greys_r")
                axs[i * nns.shape[1] + j][k].axis("off")
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()

# Just k-NN

In [ ]:
X_train_ravelled = np.array([row.ravel() for row in X_train])
X_train = X_train_ravelled

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

In [ ]:
knn.score(X_train, y_train)

# Just k-means

In [ ]:
from sklearn.cluster import KMeans

X_train = X_train[:5000]
y_train = y_train[:5000]

kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(X_train)

def cluster_acc(Y_pred, Y):
    """
    Finds the cluster accuracy
    """
    from sklearn.utils.linear_assignment_ import linear_assignment
    Y_pred = np.array(Y_pred)
    Y = np.array(Y)
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in xrange(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w

y_pred = kmeans.predict(X_train)
cluster_acc(y_pred, y_train)

# Just t-SNE

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(X_train)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_tsne, y_train)

In [ ]:
knn.score(X_tsne, y_train[:X_tsne.shape[0]])